In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"devanshabrol","key":"67d42e9299c78918a41916fac8616c98"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:10<00:00, 267MB/s]
100% 2.29G/2.29G [00:10<00:00, 236MB/s]


In [ ]:
! unzip chest-xray-pneumonia.zip

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image

In [ ]:
train_folder = 'chest_xray/train'
test_folder = 'chest_xray/test'
val_folder = 'chest_xray/val'

In [ ]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    train_folder,
    labels="inferred",
    label_mode="int",  # categorical, binary
    # class_names=['0', '1', '2', '3', ...]
    color_mode="rgb",
    batch_size=32,
    image_size=(64, 64),  # reshape if not in this size
    shuffle=True,
    seed=123,
)
def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return image, y


ds_train = ds_train.map(augment)

Found 5216 files belonging to 2 classes.


In [ ]:
import os
print(len(os.listdir(train_folder+'/NORMAL')) + len(os.listdir(train_folder+'/PNEUMONIA')))

5216


In [ ]:
ds_test = tf.keras.preprocessing.image_dataset_from_directory(
    test_folder,
    labels="inferred",
    label_mode="int",  # categorical, binary
    # class_names=['0', '1', '2', '3', ...]
    color_mode="rgb",
    batch_size=32,
    image_size=(64, 64),  # reshape if not in this size
    shuffle=True,
    seed=123,
)
def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return image, y


ds_train = ds_train.map(augment)

Found 624 files belonging to 2 classes.


In [ ]:
ds_test

<BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
model = keras.Sequential(
    [
     keras.Input(shape=(64,64,3)),
     layers.Conv2D(32,3,padding='valid',activation='relu'),
     layers.MaxPool2D(),
     layers.Conv2D(64,3,padding='valid',activation='relu'),
     layers.MaxPool2D(),
     layers.Conv2D(128,3,padding='valid',activation='relu'),
     layers.Flatten(),
     layers.Dense(64, activation='relu'),
     layers.Dense(1,activation='sigmoid'),
    ]
)
model.compile(
    loss = keras.losses.binary_crossentropy,
    optimizer = keras.optimizers.Adam(lr=3e-4),
    metrics = ['accuracy'],
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 flatten (Flatten)           (None, 18432)             0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(ds_train, batch_size=32, epochs=10, verbose=2)
model.evaluate(ds_test, batch_size=32, verbose=2)

Epoch 1/10
163/163 - 33s - loss: 1.0666 - accuracy: 0.8967 - 33s/epoch - 204ms/step
Epoch 2/10
163/163 - 21s - loss: 0.0946 - accuracy: 0.9653 - 21s/epoch - 128ms/step
Epoch 3/10
163/163 - 21s - loss: 0.0671 - accuracy: 0.9770 - 21s/epoch - 126ms/step
Epoch 4/10
163/163 - 21s - loss: 0.0471 - accuracy: 0.9839 - 21s/epoch - 127ms/step
Epoch 5/10
163/163 - 20s - loss: 0.0369 - accuracy: 0.9856 - 20s/epoch - 126ms/step
Epoch 6/10
163/163 - 21s - loss: 0.0275 - accuracy: 0.9895 - 21s/epoch - 126ms/step
Epoch 7/10
163/163 - 21s - loss: 0.0177 - accuracy: 0.9946 - 21s/epoch - 131ms/step
Epoch 8/10
163/163 - 21s - loss: 0.0166 - accuracy: 0.9954 - 21s/epoch - 128ms/step
Epoch 9/10
163/163 - 21s - loss: 0.0163 - accuracy: 0.9960 - 21s/epoch - 126ms/step
Epoch 10/10
163/163 - 21s - loss: 0.0087 - accuracy: 0.9977 - 21s/epoch - 127ms/step
20/20 - 2s - loss: 1.2322 - accuracy: 0.7981 - 2s/epoch - 110ms/step


[1.2321680784225464, 0.7980769276618958]

In [ ]:
model.save('chest_xray_cnn')

In [ ]:
!zip -r file.zip chest_xray_cnn

  adding: chest_xray_cnn/ (stored 0%)
  adding: chest_xray_cnn/variables/ (stored 0%)
  adding: chest_xray_cnn/variables/variables.index (deflated 66%)
  adding: chest_xray_cnn/variables/variables.data-00000-of-00001 (deflated 44%)
  adding: chest_xray_cnn/assets/ (stored 0%)
  adding: chest_xray_cnn/keras_metadata.pb (deflated 91%)
  adding: chest_xray_cnn/saved_model.pb (deflated 89%)


In [ ]:
img = Image.open(val_folder+'/NORMAL/'+'NORMAL2-IM-1427-0001.jpeg')

In [ ]:
!mkdir 

In [ ]:
!mkdir inputfile

In [ ]:
! cp chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg inputfile/image

In [ ]:
ds_eval = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/inputfile',
    labels="inferred",
    label_mode="int",  # categorical, binary
    # class_names=['0', '1', '2', '3', ...]
    color_mode="rgb",
    image_size=(64, 64),  # reshape if not in this size
)

Found 1 files belonging to 2 classes.


In [ ]:
res = model.predict(ds_eval)
if (res[0][0]>0.5):
  print('pneumonia')
else:
  print('normal')

pneumonia
